In [1]:
import re
import csv
import pandas as pd

In [30]:
### CONSTANTS ###
DATA_LINE = 15
ORIGINAL_FILENAME = "wine_quality.csv"
FILENAME = "wine_quality_cleaned.csv"

In [31]:
### CLEAN CSV FILE - ONLY RUN ONCE TO GET CLEANED CSV ###
file = open('{}'.format(ORIGINAL_FILENAME), "r")

fieldnames = []
for i in range(0, DATA_LINE-1):
    line = file.readline()
    m = re.search('(?<=@attribute )(.+)(?= \w+)', line)
    if (m):
        fieldnames.append(m.group(0))

with open('{}'.format(FILENAME), "w", newline="") as clean_csv:
    clean_csv.write(','.join(fieldnames) + "\n")
    for line in file:
        clean_csv.write(file.readline())

file.close()

In [33]:
# data = pd.read_csv('{}'.format(FILENAME),sep="delimiter",header=None,engine='python')
data = pd.read_csv
data

,0
0,"fixedacid,volacid,citricacid,residualsugar,chl..."
1,"7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4..."
2,"7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6"
3,"8.1,0.22,0.43,1.5,0.044,28,129,0.9938,3.22,0.4..."
4,"8.6,0.23,0.4,4.2,0.035,17,109,0.9947,3.14,0.53..."
...,...
2446,"6.5,0.33,0.38,8.3,0.035,60,148,0.98964,3.27,0...."
2447,"6.6,0.34,0.4,8.1,0.046,68,170,0.99494,3.15,0.5..."
2448,"6.8,0.22,0.36,1.2,0.052,38,127,0.9933,3.04,0.5..."
2449,"6.1,0.34,0.29,2.2,0.036,25,100,0.98938,3.06,0...."
